In [1]:
# 추출한 데이터 불러오기
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np

cats = {
    "반팔": "001001",
    "맨투맨": "001005",
    "후드집업": "002022",
    "숏패딩": "002012",
    "데님팬츠": "003002",
    "코튼팬츠": "003007",
}
selected_cat = "맨투맨"

# Encode categorical variables (gender) using LabelEncoder
label_encoder = LabelEncoder()
# Normalize numerical features
scaler = StandardScaler()

dir = "datasets/"
file_name = dir + selected_cat + "_data.csv"
data = pd.read_csv(file_name)
data = data[["총장", "어깨너비", "가슴단면", "소매길이", "성별", "키", "몸무게", "후기"]].dropna(axis=0)
#data = data[["총장", "허리단면", "엉덩이단면", "허벅지단면", "밑위", "밑단단면", "성별", "키", "몸무게", "후기"]].dropna(axis=0)
print(data)

         총장  어깨너비  가슴단면  소매길이  성별    키  몸무게   후기
0      67.0  52.0  57.0  58.0   0  159   56  적당함
1      67.0  52.0  57.0  58.0   0  163   47    큼
2      67.0  52.0  57.0  58.0   0  163   52  적당함
3      67.0  52.0  57.0  58.0   0  160   42    큼
4      67.0  52.0  57.0  58.0   0  162   55  적당함
...     ...   ...   ...   ...  ..  ...  ...  ...
18136  74.0  53.0  61.0  67.0   1  178   76  적당함
18137  74.0  53.0  61.0  67.0   1  179   78  적당함
18138  74.0  53.0  61.0  67.0   1  181   70  적당함
18139  74.0  53.0  61.0  67.0   1  186   65  적당함
18140  74.0  53.0  61.0  67.0   1  186   65  적당함

[16453 rows x 8 columns]


In [2]:
# 후기를 제외한 열은 입력 데이터로 사용
# 후기는 타깃 데이터
#총장,허리단면,엉덩이단면,허벅지단면,밑위,밑단단면,성별,키,몸무게,후기

data_input = data[["총장", "어깨너비", "가슴단면", "소매길이", "성별", "키", "몸무게"]].to_numpy()
#data_input = data[["총장", "허리단면", "엉덩이단면", "허벅지단면", "밑위", "밑단단면", "성별", "키", "몸무게"]].to_numpy()
data_target = data["후기"].to_numpy()

rv2num = {"큼": 2, "적당함": 1, "작음": 0}
for i in range(len(data_target)):
    data_target[i] = rv2num[data_target[i]]
data_target = data_target.astype(np.int64)

In [3]:
# 사이킷런의 train_test_split() 함수를 이용해 이 데이터를 훈련 세트와 테스트 세트로 나눔
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data_input, data_target, random_state=42
)

data_input = scaler.fit_transform(data_input)
test_input = scaler.transform(test_input)

In [44]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_dim=data_input.shape[1]),
    tf.keras.layers.Dropout(0.5),  # Corrected: Use a comma instead of a semicolon
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: small, moderate, large
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Corrected: Use sparse_categorical_crossentropy

# Train the model
model.fit(train_input, train_target, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_input, test_target)
print(f"Test Accuracy: {accuracy}")

# Make predictions
predictions = model.predict(test_input)

Epoch 1/20
309/309 [==============================] - 2s 7ms/step - loss: 5.4007 - accuracy: 0.7050 - val_loss: 1.6474 - val_accuracy: 0.7958
Epoch 2/20
309/309 [==============================] - 2s 6ms/step - loss: 1.3897 - accuracy: 0.7149 - val_loss: 0.7549 - val_accuracy: 0.8079
Epoch 3/20
309/309 [==============================] - 2s 6ms/step - loss: 1.1629 - accuracy: 0.7282 - val_loss: 0.6917 - val_accuracy: 0.8063
Epoch 4/20
309/309 [==============================] - 2s 6ms/step - loss: 1.0604 - accuracy: 0.7289 - val_loss: 0.9809 - val_accuracy: 0.7978
Epoch 5/20
309/309 [==============================] - 2s 6ms/step - loss: 0.9281 - accuracy: 0.7369 - val_loss: 0.8654 - val_accuracy: 0.7962
Epoch 6/20
309/309 [==============================] - 2s 6ms/step - loss: 0.9430 - accuracy: 0.7360 - val_loss: 0.7735 - val_accuracy: 0.7581
Epoch 7/20
309/309 [==============================] - 2s 6ms/step - loss: 0.8537 - accuracy: 0.7447 - val_loss: 0.7075 - val_accuracy: 0.7986
Epoch 

In [70]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

# 표를 불러오기 위한 라이브러리
from html_table_parser import parser_functions as parser
import collections

collections.Callable = collections.abc.Callable
# 403 에러 방지
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# https://www.musinsa.com/app/goods/3574305
# 0, 180, 80
url = input("url 주소를 입력하세요")
# Collect additional user information (gender, height, weight)
user_gender = input("Enter your gender (if 'Male' 0, if 'Female' 1): ")
user_height = float(input("Enter your height in centimeters: "))
user_weight = float(input("Enter your weight in kilograms: "))

In [71]:
user_info = {"성별": [user_gender], "키":[user_height], "몸무게":[user_weight]}
user_info = pd.DataFrame(user_info)

In [72]:
page = requests.get(url, headers=headers)
soup = bs(page.text, "lxml")
table = soup.find_all(attrs={"class": "table_th_grey"})

p = parser.make2d(table[0])
p[0][0] = "사이즈"
size_df = pd.DataFrame(p[3:], columns=p[0])

num_rows = len(size_df)
user_info = pd.concat([user_info] * num_rows, ignore_index=True)
user_df = pd.concat([size_df, user_info], axis = 1)
user_df=user_df[["총장", "어깨너비", "가슴단면", "소매길이", "성별", "키", "몸무게"]].dropna(axis=0)

print(user_df)

   총장 어깨너비 가슴단면 소매길이 성별      키   몸무게
0  61   49   53   58  0  180.0  80.0
1  68   52   58   60  0  180.0  80.0
2  71   55   60   62  0  180.0  80.0
3  74   57   65   64  0  180.0  80.0
4  79   65   70   65  0  180.0  80.0
5  82   68   75   66  0  180.0  80.0


In [73]:
# Preprocess the user's input
# Ensure the column order and preprocessing steps match your training data preprocessing
user_df=user_df[["총장", "어깨너비", "가슴단면", "소매길이", "성별", "키", "몸무게"]].dropna(axis=0)
print(user_df)
user_df[["총장", "어깨너비", "가슴단면", "소매길이", "성별", "키", "몸무게"]] = scaler.transform(user_df[["총장", "어깨너비", "가슴단면", "소매길이", "성별", "키", "몸무게"]])

# Make predictions for each row
user_predictions = model.predict(user_df.to_numpy())

print(user_predictions)

# Get the predicted class for each row
predicted_classes = np.argmax(user_predictions, axis=1)
predicted_probablities = np.max(user_predictions, axis=1)

# Map the predicted classes back to size satisfaction categories
num2rv = {2: "large", 1: "good", 0: "small"}
predicted_size_satisfactions = [num2rv[predicted_class] for predicted_class in predicted_classes]

# Add the predicted size satisfaction as a new column to the user_input DataFrame
user_df["Predicted Size Satisfaction"] = predicted_size_satisfactions
user_df["Predicted Probabilities"] = predicted_probablities


# Display the results
for i in range(num_rows):
    print(size_df['사이즈'][i], user_df['Predicted Size Satisfaction'][i], user_df['Predicted Probabilities'][i])

   총장 어깨너비 가슴단면 소매길이 성별      키   몸무게
0  61   49   53   58  0  180.0  80.0
1  68   52   58   60  0  180.0  80.0
2  71   55   60   62  0  180.0  80.0
3  74   57   65   64  0  180.0  80.0
4  79   65   70   65  0  180.0  80.0
5  82   68   75   66  0  180.0  80.0
1/1 [==============================] - 0s 130ms/step
[[0.39991853 0.37720445 0.22287704]
 [0.33984292 0.38156816 0.27858898]
 [0.31036392 0.37966463 0.3099714 ]
 [0.26809487 0.37417218 0.35773298]
 [0.21531238 0.3606924  0.42399517]
 [0.18046397 0.345928   0.47360805]]
M small 0.39991853
L good 0.38156816
XL good 0.37966463
2XL good 0.37417218
3XL large 0.42399517
4XL large 0.47360805


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
